In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.base import TaskResult
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import TextMessage
from autogen_core.tools import FunctionTool
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

model_client = OpenAIChatCompletionClient(model="gpt-4o-mini",api_key=api_key)

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper = api_wrapper_wiki)

## Tools

In [3]:
def wikipedia_search(query:str)->str:
    """
    Used to find maths formula and its theory.
    """
    try:
        wiki.invoke(query)
    except Exception as e:
        print(f"Error Happened : {str(e)}")

web_search_tool = FunctionTool(func=wikipedia_search,description="Used to find maths formula and its theory.")

## Agents

In [4]:
wikipedia_agent = AssistantAgent(
    name="Web_Search",
    model_client=model_client,
    system_message="""
    You are a factual information retrieval assistant.

    Your job is to search Wikipedia or reliable sources to extract concise, accurate, and relevant facts or definitions based on the user’s query. Do not perform any reasoning or calculations. Only provide factual information or summaries from your search results.

    If a search term is ambiguous, try to provide the most common or relevant interpretation. Include dates, formulas, or names only if directly related.

    Always return the facts in a clean, bullet-pointed or paragraph form, and let the next agent handle reasoning or calculation.
    """,
    tools=[web_search_tool],
    reflect_on_tool_use=True
)

calculator_agent = AssistantAgent(
    name="Calculator",
    model_client=model_client,
    system_message="""
    You are a calculator and numerical problem-solver.

    Your job is to perform precise mathematical calculations based on the input provided. Use Python code if needed to solve equations, evaluate expressions, or compute results from provided data.

    Do not attempt to reason, interpret, or analyze concepts — just return accurate computations.
    """
)

reasoning_agent = AssistantAgent(
    name="Reasoner",
    model_client=model_client,
    system_message="""
    You are a reasoning and explanation agent.

    You will receive facts (from a search agent) and numerical results (from a calculator agent). Your task is to analyze, draw logical conclusions, and answer the user’s original question with clarity.

    Use structured thinking and explain the final answer step-by-step, if appropriate. Be concise, avoid repeating earlier outputs, and ensure the logic flows clearly from the given data.

    ALWAYS AFTER FINISHING YOU HAVE TO SAY "STOP"
    """
)

In [5]:
team = RoundRobinGroupChat(
    participants=[wikipedia_agent,calculator_agent,reasoning_agent],
    termination_condition = TextMentionTermination("STOP"),
    max_turns = 15
)

In [6]:
async def main():
    task = TextMessage(content="What is the distance a freely falling object covers in the first 5 seconds? Assume acceleration due to gravity is standard",source="user")

    await Console(team.run_stream(task=task))

await main()

---------- TextMessage (user) ----------
What is the distance a freely falling object covers in the first 5 seconds? Assume acceleration due to gravity is standard
---------- ToolCallRequestEvent (Web_Search) ----------
[FunctionCall(id='call_ICFCVuDQd4gTBe5aWd7l7elH', arguments='{"query":"distance covered by freely falling object formula"}', name='wikipedia_search')]
---------- ToolCallExecutionEvent (Web_Search) ----------
[FunctionExecutionResult(content='None', name='wikipedia_search', call_id='call_ICFCVuDQd4gTBe5aWd7l7elH', is_error=False)]
---------- TextMessage (Web_Search) ----------
The distance \( d \) covered by a freely falling object can be calculated using the formula:

\[ 
d = \frac{1}{2} g t^2 
\]

Where:
- \( g \) is the acceleration due to gravity (approximately \( 9.81 \, \text{m/s}^2 \))
- \( t \) is the time in seconds

For a time period of 5 seconds:

- \( d = \frac{1}{2} \times 9.81 \times (5)^2 \)
- \( d = \frac{1}{2} \times 9.81 \times 25 \)
- \( d = \frac{1}{